In [42]:
import pandas as pd

from faker import Faker
import faker_commerce

import json


In [43]:
## Initialize Faker
fake = Faker()
fake.add_provider(faker_commerce.Provider)

In [84]:
## Set ABC Type
### By usage or by dollar value
types = ['usage', 'dollar_value']
type = types[1]

## Creating Fake Dataset

In [75]:
## Part Profile Definition

def create_data(x):
    
    part_volume ={}
    
    for i in range(0,x):
        part_volume[i]={}
        part_volume[i]['part_number']= fake.bothify(text='????-########', letters='ABCDE')
        part_volume[i]['annual_usage']= fake.random_int(100,100000)
        part_volume[i]['average_cost'] =  fake.random_number(digits=4)/10000
        
    return part_volume

data = create_data(100)

parts = pd.DataFrame.from_dict(data).T
parts.head()

,part_number,annual_usage,average_cost
0,DBBC-46966652,41383,0.0414
1,BDCD-44132115,40180,0.5672
2,ECDB-73833922,95661,0.0505
3,EEAC-71841372,47808,0.903
4,DBDC-03155984,66052,0.9427


## ABC Analysis

In [76]:
## Calculate Total Annual Value

parts['annual_value'] = parts['annual_usage'] * parts['average_cost']

In [77]:
## Sort by total value

parts.sort_values('annual_value', ascending=False, inplace=True)
parts.head()

,part_number,annual_usage,average_cost,annual_value
54,ECBA-26995570,92274,0.9916,91498.8984
43,DDAE-31338460,99287,0.9151,90857.5337
69,DCAD-97209053,92500,0.9482,87708.5
27,EAED-06023177,87362,0.8968,78346.2416
70,BCEB-00471624,96640,0.801,77408.64


In [85]:
## Calculate Accumulating Sum

total_value = sum(parts.annual_value)
total_usage = sum(parts.annual_usage)

if type == 'dollar_value':
    parts['percent'] = (parts.annual_value/total_value)*100
else:
    parts['percent'] = (parts.annual_usage/total_usage)*100

parts['accumulating_sum'] = parts.percent.cumsum()

In [87]:
## Categorize into A, B, or C

def categorize(x):
    if x <= 20:
        return 'A'
    elif x <= 95:
        return 'B'
    else:
        return 'C'

parts['category'] = parts.accumulating_sum.apply(categorize)
parts.head(15)

,part_number,annual_usage,average_cost,annual_value,percent,accumulating_sum,category
54,ECBA-26995570,92274,0.9916,91498.8984,3.701501,3.701501,A
43,DDAE-31338460,99287,0.9151,90857.5337,3.675555,7.377057,A
69,DCAD-97209053,92500,0.9482,87708.5,3.548164,10.925221,A
27,EAED-06023177,87362,0.8968,78346.2416,3.169423,14.094644,A
70,BCEB-00471624,96640,0.801,77408.64,3.131493,17.226137,A
68,ABCC-49459875,89487,0.857,76690.359,3.102436,20.328573,B
51,ABDD-66992394,88235,0.8197,72326.2295,2.925889,23.254462,B
28,DABE-81431600,95136,0.6743,64150.2048,2.595136,25.849597,B
40,BCDE-28034787,68426,0.9325,63807.245,2.581262,28.430859,B
4,DBDC-03155984,66052,0.9427,62267.2204,2.518961,30.94982,B
